In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from os import system

import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve,classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
path = "../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"

df=pd.read_csv(path)
df.head()

In [ ]:
#Shape of the data
df.shape

In [ ]:
#Null Check
df.isnull().sum()

In [ ]:
#Duplicate CHeck
df.duplicated().sum()

In [ ]:
#Column Level Information
df.info()

In [ ]:
#Dropping 11 records - Total Charges has Blank values
df[df.TotalCharges==' '].TotalCharges

In [ ]:
df=df[df.TotalCharges!=' ']
df.shape

In [ ]:
#Object data type to Float Conversion for TotalCharges column
df.TotalCharges=df.TotalCharges.astype('float64')
print (df.dtypes)

In [ ]:
# 5 Point summary on the data
df.describe()

In [ ]:
#Dependent variable data distribution
df.Churn.value_counts()

In [ ]:
df.Churn.value_counts(normalize=True)

In [ ]:
plot = df.Churn.value_counts().plot.pie(y='Churn', figsize=(5, 5), autopct='%1.0f%%')

In [ ]:
df.columns[df.dtypes!='object']

In [ ]:
df.columns[df.dtypes=='object']

In [ ]:
#Categorical Unique values:
for i in df.columns[df.dtypes=='object']:
    print((i),':')
    print(df[i].value_counts())

In [ ]:
#Data distribution of each columns
data_plot=df[df.columns[df.dtypes!='object']]
fig=plt.figure(figsize=(20,30))
for i in range(0,len(data_plot.columns)):
    ax=fig.add_subplot(6,3,i+1)
    skew =data_plot[data_plot.columns[i]].skew()
    sns.distplot(data_plot[data_plot.columns[i]],hist=True,label='Skew = %.3f' %(skew))
    ax.set_title(data_plot.columns[i],color='Red')
    plt.legend(loc='best')
plt.tight_layout()
plt.show()

Total Charges : Right Skewed

Tenure is distribued over 0 to 80 months

Monthly charges are varying from 20 to 120  and highest data concentration is near 20 to 40

Total Charges - data is varying from 0 to 10000 and right skewed

In [ ]:
data_plot=df[df.columns[df.dtypes!='object']]
col=df.columns[df.dtypes!='object']
fig=plt.figure(figsize=(20,20))
for i in range(0,len(data_plot.columns)):
    ax=fig.add_subplot(5,4,i+1)
    sns.stripplot(df.Churn,df[col[i]],jitter=True)
    ax.set_title(data_plot.columns[i],color='Red')
plt.tight_layout()
plt.show()

Monthly Charges of Churned customers are varing in lower range of 20 to 60

Total charges of churned customers having density near 0-4000

In [ ]:
data_plot=df[df.columns[df.dtypes=='object']]
col=df.columns[df.dtypes=='object']
fig=plt.figure(figsize=(20,20))
for i in range(0,len(data_plot.columns)):
    ax=fig.add_subplot(5,4,i+1)
    sns.stripplot(df.Churn,df[col[i]],jitter=True)
    ax.set_title(data_plot.columns[i],color='Red')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.pairplot(df,hue='Churn' , diag_kind = 'kde')

Tenure and Total Charges shows very good correlation from the graph

Monthly charges and tenure also having little good concentration

In [ ]:
corr=df.corr()
corr

In [ ]:
plt.figure(figsize=(25,25)) 
p=sns.heatmap(corr, annot=True,square=True)

In [ ]:
df.columns[df.dtypes=='object']

In [ ]:
col=df.columns[df.dtypes=='object']
fig=plt.figure(figsize=(20,15))
for i in range(0,len(col)):
    ax=fig.add_subplot(6,3,i+1)
    sns.countplot(x=col[i], hue='Churn', data=df)
    ax.set_title(data_plot.columns[i],color='Green')
plt.tight_layout()
plt.show()

Gender concentration - 50-50

Customers not having partner have churned more than having partners

Customers not having dependent have churned most

Customers having phone services have churned most 

Customers having multiple lines and fibre obtip services have churned more

Customers have churned more when they are not given with tech support

Customers having month to month contract have chruned most

Customers having paperless billing, electronic check payments have churned most

In [ ]:
#Categorical Encoding
for feature in df.columns: 
    if df[feature].dtype == 'object': 
        print('\n')
        print('feature:',feature)
        print(pd.Categorical(df[feature].unique()))
        print(pd.Categorical(df[feature].unique()).codes)
        df[feature] = pd.Categorical(df[feature]).codes

In [ ]:
#Dropping columns which are not required from dataset
df=df.drop('customerID', axis=1)
df.head(1)

In [ ]:
#Outlier Check
data_plot=df[df.columns[df.dtypes!='object']]

In [ ]:
fig=plt.figure(figsize=(20,20))
for i in range(0,len(data_plot.columns)):
    ax=fig.add_subplot(5,4,i+1)
    sns.boxplot(x='Churn',y=data_plot[data_plot.columns[i]],data=data_plot)
    ax.set_title(data_plot.columns[i],color='Green')
plt.tight_layout()
plt.show()

In [ ]:
fig=plt.figure(figsize=(20,20))
for i in range(0,len(data_plot.columns)):
    ax=fig.add_subplot(5,4,i+1)
    sns.boxplot(data_plot[data_plot.columns[i]])
    ax.set_title(data_plot.columns[i],color='Green')
plt.tight_layout()
plt.show()

 # **Train and Test Split**

In [ ]:
# Copy all the predictor variables into X dataframe
X = df.drop(['Churn'], axis=1)

# Copy target into the y dataframe.  
y = df['Churn']

In [ ]:
# Split X and y into training and test set in 70:30 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() 
X_train = sc.fit_transform(X_train) 
X_test = sc.transform (X_test)

In [ ]:
mlp=MLPClassifier(hidden_layer_sizes=(200),random_state=1)

In [ ]:
mlp.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mlp, X_train, y_train, cv=10)
scores

In [ ]:
## Performance Matrix on train data set
y_train_predict = mlp.predict(X_train)
model_score =mlp.score(X_train, y_train)
print(model_score)
print(metrics.confusion_matrix(y_train, y_train_predict))
print(metrics.classification_report(y_train, y_train_predict))

#SENSITIVITY/RECALL=TP/TP+FN

#SPECIFICITY=TN/TN+FP

#ACCURACY=(TP+TN)/(TP+TN+FP+FN)

#PRECISION=TP/TP+FP


In [ ]:
# predict probabilities
probs = mlp.predict_proba(X_train)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(y_train, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(train_fpr, train_tpr)

In [ ]:
ANN_metrics=classification_report(y_train, y_train_predict,output_dict=True)
df1=pd.DataFrame(ANN_metrics).T
ANN_train_precision=round(df1.loc["1"][0],2)
ANN_train_recall=round(df1.loc["1"][1],2)
ANN_train_f1=round(df1.loc["1"][2],2)
ANN_train_acc=round(df1.loc["accuracy"][0],2)
print ('ANN_train_precision ',ANN_train_precision)
print ('ANN_train_recall ',ANN_train_recall)
print ('ANN_train_f1 ',ANN_train_f1)
print('ANN_train_acc',ANN_train_acc)

In [ ]:
## Performance Matrix on test data set
y_test_predict = mlp.predict(X_test)
model_score = mlp.score(X_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_test_predict))
print(metrics.classification_report(y_test, y_test_predict))

In [ ]:
# predict probabilities
probs = mlp.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
test_auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(test_fpr, test_tpr)

In [ ]:
ANN_metrics=classification_report(y_test, y_test_predict,output_dict=True)
df1=pd.DataFrame(ANN_metrics).T
ANN_test_precision=round(df1.loc["1"][0],2)
ANN_test_recall=round(df1.loc["1"][1],2)
ANN_test_f1=round(df1.loc["1"][2],2)
ANN_test_acc=round(df1.loc["accuracy"][0],2)
print ('ANN_test_precision ',ANN_test_precision)
print ('ANN_test_recall ',ANN_test_recall)
print ('ANN_test_f1 ',ANN_test_f1)
print('ANN_test_acc',ANN_test_acc)

# MODEL TUNING

In [ ]:
param_grid = {
    'hidden_layer_sizes': [200,300], 
    'max_iter': [5000],
    'solver': ['adam'], 
    'tol': [0.0001], 
}

nncl = MLPClassifier()

grid_search = GridSearchCV(estimator = nncl, param_grid = param_grid, cv =5)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
best_grid

In [ ]:
## Performance Matrix on train data set
y_train_predict = best_grid.predict(X_train)
model_score =best_grid.score(X_train, y_train)
print(model_score)
print(metrics.confusion_matrix(y_train, y_train_predict))
print(metrics.classification_report(y_train, y_train_predict))

In [ ]:
# predict probabilities
probs = best_grid.predict_proba(X_train)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(y_train, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(train_fpr, train_tpr)

In [ ]:
ANN_metrics=classification_report(y_train, y_train_predict,output_dict=True)
df1=pd.DataFrame(ANN_metrics).T
ANN_train_precision_M=round(df1.loc["1"][0],2)
ANN_train_recall_M=round(df1.loc["1"][1],2)
ANN_train_f1_M=round(df1.loc["1"][2],2)
ANN_train_acc_M=round(df1.loc["accuracy"][0],2)
print ('ANN_train_precision_M ',ANN_train_precision_M)
print ('ANN_train_recall_M ',ANN_train_recall_M)
print ('ANN_train_f1_M ',ANN_train_f1_M)
print('ANN_train_acc_M',ANN_train_acc_M)

In [ ]:
## Performance Matrix on test data set
y_test_predict = best_grid.predict(X_test)
model_score = best_grid.score(X_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_test_predict))
print(metrics.classification_report(y_test, y_test_predict))

In [ ]:
# predict probabilities
probs = best_grid.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
test_auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(test_fpr, test_tpr)

In [ ]:
ANN_metrics=classification_report(y_test, y_test_predict,output_dict=True)
df1=pd.DataFrame(ANN_metrics).T
ANN_test_precision_M=round(df1.loc["1"][0],2)
ANN_test_recall_M=round(df1.loc["1"][1],2)
ANN_test_f1_M=round(df1.loc["1"][2],2)
ANN_test_acc_M=round(df1.loc["accuracy"][0],2)
print ('ANN_test_precision_M ',ANN_test_precision_M)
print ('ANN_test_recall_M ',ANN_test_recall_M)
print ('ANN_test_f1_M ',ANN_test_f1_M)
print('ANN_test_acc_M',ANN_test_acc_M)

#Insights from EDA:

*Dependent variable concentration is 73:27 percentage
    
* Most of the Customers churn happened because of not getting the technical support
    
*Customer churn for Male and Female customers are almost 50:50 percent
    
*Tenure is distribued over 0 to 80 months
    
* Monthly charges are varying from 20 to 120  and highest data concentration is near 20 to 40
    
*Total Charges - data is varying from 0 to 10000 and right skewed
    
*Tenure and Total Charges shows very good correlation from the graph
    
*Customers not having partner have churned more than having partners
    
*Customers not having dependent have churned most
    
*Customers having phone services have churned most 
    
*Customers having multiple lines and fibre obtip services have churned more
    
*Customers have churned more when they are not given with tech support
    
*Customers having month to month contract have chruned most
    
*Customers having paperless billing, electronic check payments have churned most